## YOLO - You Only Look Once 

In [ ]:
import cv2
import numpy as np
import functools
import time
import os.path

### Pobieranie wag:

In [ ]:
if not os.path.isfile('yolov3.weights'):
    !wget https://pjreddie.com/media/files/yolov3.weights

In [ ]:
if not os.path.isfile('yolov3-tiny.weights'):
    !wget https://pjreddie.com/media/files/yolov3-tiny.weights

### Wczytanie wag do modelu dostarczonego przez OpenCv.

Do wyboru wagi oryginalne oraz wagi modelu uproszczonego - tiny-yolo, który kosztem jakości detekcji znacząco przyspiesza inferencję

In [ ]:
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg") # Original yolov3
# net = cv2.dnn.readNet("yolov3-tiny.weights", "yolov3-tiny.cfg") #Tiny Yolo

layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

Wypisanie nazw klas obiektów rozpoznawanych przez model

In [ ]:
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
print(classes)

### Funkcje pomocnicze

In [ ]:
def prapagate_input(frame):
    blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False)
    net.setInput(blob)
    return net.forward(outputlayers)

In [ ]:
def get_frame(video_capture, skip=5):
    # pobieranie jedenj klatki na kilka, aby uniknąć opóźnień
    for i in range(skip):
        video_capture.read()
    _,frame= video_capture.read()
    return frame

In [ ]:
def get_bounding_boxes(inference_results, width, height):
    class_ids=[]
    confidences=[]
    boxes=[]
    for result in inference_results:
        for detection in result:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.4:
                #skalowanie znalezionych współrzędnych do przetwarzanego obrazu                 
                center_x= int(detection[0]*width)
                center_y= int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                #współrzędne prostokąta (bounding box'a)
                x=int(center_x - w/2)
                y=int(center_y - h/2)

                boxes.append([x,y,w,h]) 
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    return class_ids, confidences, boxes

In [ ]:
def draw_bounding_boxes(boxes, indexes):
    colors= np.random.uniform(0,255,size=(len(classes),3))
    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence= confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
            cv2.putText(frame,label+" "+str(round(confidence,2)),(x,y+30),font,1,(255,255,255),2)

In [ ]:
def put_fps_on_image(start_time):
    elapsed_time = time.time() - start_time
    fps=frame_id/elapsed_time
    cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,0),1)

### Przekształcenia obrazu

In [ ]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)

In [ ]:
def rotate_90(img):
    return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

In [ ]:
def flip(img):
    return cv2.flip(img, 1)

In [ ]:
def negative(img):
    return cv2.bitwise_not(img)

In [ ]:
def blur(img):
    return cv2.blur(img,(20,20))

In [ ]:
def apply_transformations(transformations, img):
    for transformation in transformations:
        img = transformation(img)
    return img

### Zachęca się do przeprowadzenia eksperymentów, sprawdzających jak zachowa się model, gdy jako wejście otrzyma przekształcony obraz

Należy odkomentować interesujące nas transformacje

In [ ]:
transformations = [
#     grayscale,
#     rotate_90,
#     flip,
#     negative,
#     blur
]

### Uruchomienie detekcji obrazów z kamery za pomocą Yolo 
Aby zamknąć okno należy wcisnąć ESC

In [ ]:
#inicjalizacja kamerki
video_capture = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_PLAIN
start_time = time.time()
frame_id = 0

while True:
    #Pobranie klatki z kamery     
    frame = get_frame(video_capture)
    frame = apply_transformations(transformations, frame)
    frame_id+=1
    height,width,channels = frame.shape
    #Detekcja za pomocą Yolo
    inference_results = prapagate_input(frame)

    #Podział wynikowego wektora inferencji na bounding boxy
    class_ids, confidences, boxes = get_bounding_boxes(inference_results, width, height)

    # Użycie Non-Max-Supression aby zostawić tylko jeden bounding box per obiekt
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.6)

    draw_bounding_boxes(boxes, indexes)  
    put_fps_on_image(start_time)
    
    cv2.imshow("OpenCv_Yolov3",frame)
    key = cv2.waitKey(1)
    
    #ESC
    if key == 27: 
        break
    
video_capture.release()    
cv2.destroyAllWindows()

### Porównanie Yolo vs tiny-Yolo

Wiersz jakość detekcji należy uzupełnić subiektywną opinią z przedziału [bardzo dobra, dobra, średnia, zła, bardzo zła] 

|                       |**Yolo**       |**tiny-Yolo** 	|
| :-----------          | ------------- | ------------- |
| **Fps**               | todo  | todo  |
| **Rozmiar pliku wag** | todo  | todo  |
| **Jakość detekcji**   | todo  | todo  |